<a href="https://colab.research.google.com/github/dunomi/dunomi/blob/master/USA_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/virus_cases.csv')
USA_state_names= pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/states.csv')
USA_humidity_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_humidity.csv?token=AO3BJXHYBDBENHN2JIXXFGC6POTP2')
USA_temperature_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_temperature.csv?token=AO3BJXFW6K5G6GYWHX5I2GC6POTVA')

In [0]:
USA=ConfirmedCases_raw[ConfirmedCases_raw['Country/Region']=='US'].reset_index().drop(['index','Country/Region','Lat','Long'],axis=1)
#USA.sum(axis = 0, skipna = True)[1:]>10
USA=USA.head(52)
USA=USA[~USA['Province/State'].isin(['Diamond Princess','Grand Princess'])].reset_index().drop('index',axis=1)


Przygotowanie tabeli z przypadkami Corona virus

In [0]:
USAcases=pd.DataFrame(columns=['Stan','Przyrosty','Data'])

for i in range(len(USA)):
  Date=USA.loc[i][USA.loc[i][0:]!=0].index[1]
  stateName=USA.loc[i][0]
  stateValue=USA.loc[i][1:].tolist()
  stateSum=[i for i in stateValue if i>0]
  stateSumArray=np.array(stateSum[1:7])/np.array(stateSum[0:6])
  stateSumArray=np.array([round(i,2) for i in stateSumArray])
  USAcases=USAcases.append(pd.DataFrame([[stateName,stateSumArray,Date]],columns=['Stan','Przyrosty','Data']))

USAcases=USAcases.reset_index().drop('index',axis=1)[['Stan','Data','Przyrosty']]


Przygotowanie tabeli z temperaturą i wilgotnością.

USAdata - tabela z przyrostem przypadków zachorowań, stopniami temperatury oraz procentową wilgotnością.

USAdata2 - tabela z przyrostem przypadków zachorowań, przyrostem temperatury oraz przyrostem wilgotności.

In [0]:
USAweather=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][3:])
  wilg=np.array(USA_humidity_row.loc[i][3:])/100

  USAweather=USAweather.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))
  
USAdata=pd.merge(USAcases,USAweather)

In [0]:
#USAweather2=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])
#
#for i in range(len(USA_temperature_row)):
#  stateName=USA_temperature_row.loc[i][0]
#  temp=np.array(USA_temperature_row.loc[i][4:])/np.array(USA_temperature_row.loc[i][3:-1])
#  wilg=np.array(USA_humidity_row.loc[i][4:])/np.array(USA_humidity_row.loc[i][3:-1])
#
#  USAweather2=USAweather2.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))
#
#USAdata2=pd.merge(USAcases,USAweather2)

USAcumdata - tabela z danymi przygoptowanymi do wykresu dla USAdata

USAcumdata2 - tabela z danymi przygoptowanymi do wykresu dla USAdata2

In [0]:
USAcumdata=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata)):
  stateName=USAdata.loc[i][0]
  przy=np.cumprod(USAdata['Przyrosty'][i])[5]
  temp=USAdata.Temperatura[i].mean()
  wilg=USAdata.Wilgotnosc[i].mean()
  USAcumdata=USAcumdata.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

#USAcumdata.loc[USAcumdata.Przyrosty > 50, 'Przyrosty'] = max(USAcumdata.loc[USAcumdata.Przyrosty < 50].Przyrosty)

In [0]:
#USAcumdata2=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
#for i in range(0,len(USAdata2)):
#  stateName=USAdata2.loc[i][0]
#  przy=np.cumprod(USAdata2['Przyrosty'][i])[5]
#  temp=USAdata2.Temperatura[i].mean()
#  wilg=USAdata2.Wilgotnosc[i].mean()
#  USAcumdata2=USAcumdata2.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))
#
#USA2cumdataLess50=USAcumdata2[USAcumdata2.Przyrosty<50]
#USAcumdataMore50=USAcumdata[USAcumdata.Przyrosty>=50]

In [0]:
for i in range(len(USAdata)):
  USAdata.loc[i, 'Temperatura_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Temperatura[i][0:6].astype(float))[0,1]
  USAdata.loc[i, 'Wilgotnosc_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Wilgotnosc[i][0:6].astype(float))[0,1]
#print(USAdata.sort_values(by='Temperatura_cor'))


In [7]:
USAdata.sort_values(by='Temperatura_cor').reset_index().drop(columns='index')

,Stan,Data,Przyrosty,Temperatura,Wilgotnosc,Temperatura_cor,Wilgotnosc_cor
0,Oklahoma,3/10/20,"[1.0, 1.0, 1.0, 2.0, 1.75, 1.43]","[23, 24, 27, 7, 10, 7, 13]","[0.49, 0.5, 0.27, 0.91, 0.88, 0.9, 0.98]",-0.891486,0.871747
1,Louisiana,3/10/20,"[6.0, 3.17, 1.89, 2.14, 1.18, 1.49]","[24, 26, 26, 28, 27, 29, 27]","[0.77, 0.59, 0.59, 0.54, 0.53, 0.45, 0.52]",-0.811776,0.922604
2,Massachusetts,3/10/20,"[1.03, 1.14, 1.14, 1.12, 1.19, 1.2]","[19, 11, 6, 16, 12, 5, 7]","[0.62, 0.53, 0.77, 0.84, 0.32, 0.44, 0.42]",-0.768976,-0.513798
3,New York,3/10/20,"[1.27, 1.49, 1.28, 1.25, 1.39, 1.32]","[17, 8, 11, 13, 7, 8, 8]","[0.56, 0.36, 0.4, 0.63, 0.44, 0.27, 0.21]",-0.687850,-0.522874
4,Mississippi,3/12/20,"[1.0, 6.0, 1.67, 1.3, 1.62, 1.62]","[28, 24, 26, 24, 26, 27, 27]","[0.44, 0.73, 0.59, 0.6, 0.53, 0.53, 0.54]",-0.583375,0.848640
5,South Dakota,3/11/20,"[1.0, 1.0, 1.12, 1.0, 1.11, 1.1]","[19, 7, 4, -1, 2, 1, 5]","[0.24, 0.38, 0.41, 0.83, 0.81, 0.76, 0.71]",-0.438385,0.328683
6,Florida,3/10/20,"[1.87, 1.25, 1.43, 1.52, 1.51, 1.35]","[24, 25, 28, 27, 29, 29, 29]","[0.55, 0.57, 0.54, 0.56, 0.43, 0.4, 0.5]",-0.430073,0.182373
7,California,3/10/20,"[1.23, 1.25, 1.28, 1.21, 1.25, 1.31]","[19, 22, 23, 18, 14, 11, 13]","[0.45, 0.48, 0.38, 0.58, 0.71, 0.74, 0.63]",-0.346023,0.265789
8,Indiana,3/10/20,"[1.83, 1.18, 1.0, 1.23, 1.25, 1.25]","[7, 8, 14, 9, 1, 7, 8]","[0.84, 0.82, 0.8, 0.46, 0.96, 0.6, 0.64]",-0.344099,0.161694
9,Illinois,3/10/20,"[2.08, 1.28, 1.44, 1.39, 1.45, 1.13]","[5, 9, 13, 10, 3, 6, 5]","[0.82, 0.76, 0.9, 0.37, 0.88, 0.61, 0.83]",-0.261987,0.335197


In [0]:
def plot_maker(Data,Temperatura_lub_Wilgotnosc):

  fig = make_subplots(rows=10, cols=5, shared_xaxes=False,  x_title=Temperatura_lub_Wilgotnosc, y_title='Przyrosty',)
  for i in range(0,len(Data)):
    x=Data.loc[i]
    figX = px.scatter(x=x[Temperatura_lub_Wilgotnosc].tolist()[0:6],
                      y=x['Przyrosty'].tolist(),
                      trendline="ols")
    
    tracePoints = figX['data'][0]
    traceTradeline = figX['data'][1]

    fig.add_trace(tracePoints, row=math.floor(i/5)+1, col=i%5+1)
    fig.add_trace(traceTradeline, row=math.floor(i/5)+1, col=i%5+1)

    #fig.update_xaxes(title_text="xaxis 1 title", row=math.floor(i/5)+1, col=i%5+1)
    #fig.update_yaxes(title_text="yaxis 1 title", row=math.floor(i/5)+1, col=i%5+1)


  fig['layout'].update(height=1500, width=1000, showlegend=False, title=Temperatura_lub_Wilgotnosc)
  return fig.show()

plot_maker(USAdata.sort_values(by='Temperatura_cor').reset_index().drop(columns='index'),'Temperatura')
plot_maker(USAdata.sort_values(by='Wilgotnosc_cor').reset_index().drop(columns='index'),'Wilgotnosc')

Wykresy

In [0]:
fig = px.scatter(x=USAcumdata['Temperatura'].tolist(), 
                 y=USAcumdata['Wilgotnosc'].tolist(),
                 size=USAcumdata['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.update_yaxes(range=[0,1])
fig.show()


#fig = px.scatter(x=USAcumdataMore50['Temperatura'].tolist(), 
#                 y=USAcumdataMore50['Wilgotnosc'].tolist(),
#                 size=USAcumdataMore50['Przyrosty'].tolist(),
#                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
#fig.show()


In [18]:
fig1 = px.scatter(x=USAcumdata['Temperatura'][USAcumdata['Przyrosty']<100].tolist(), 
                 y=USAcumdata['Przyrosty'][USAcumdata['Przyrosty']<100].tolist(),
                 trendline="ols",
                 labels={'x':'Temperatura', 'y':'Przyrosty'})
fig1.show()

fig2 = px.scatter(x=USAcumdata['Wilgotnosc'][USAcumdata['Przyrosty']<100].tolist(), 
                 y=USAcumdata['Przyrosty'][USAcumdata['Przyrosty']<100].tolist(),
                 trendline="ols",
                 labels={'x':'Wilgotnosc', 'y':'Przyrosty'})
fig2.show()

##
fig = make_subplots(rows=1, cols=2, shared_xaxes=False,subplot_titles=("Temperatura","Wilgotność"))    

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig1['data'][1], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig2['data'][1], row=1, col=2)

fig.update_layout( yaxis_title="Regresja liniowa", bargroupgap=0.1)

fig.show()


In [0]:
USAcumdata_map=USAcumdata.reset_index()[['Stan','Przyrosty','Temperatura','Wilgotnosc']]

new_column = pd.Series([30,30,30,30], name='Przyrosty', index=USAcumdata_map[USAcumdata_map['Przyrosty']>30].index)
USAcumdata_map.update(new_column)

In [0]:
USA_us_map=USAcumdata_map.merge(USA_state_names, left_on='Stan', right_on='State')

figSubPlot = make_subplots(rows=2, cols=4, shared_xaxes=False)

fig1 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Przyrosty'], scope="usa",title='Przyrosty')
fig2 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Temperatura'], scope="usa",title='Temperatura')
fig3 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Wilgotnosc'], scope="usa",title='Wilgotnosc')

fig1.show()
fig2.show()
fig3.show()

KORELACJE

DANE DLA PRZYROSTÓW

In [0]:
# lista_temperatura=[]
# lista_wilgotnosc=[]
# for i in range(len(USAdata2)):
#   t=USAdata2.loc[i]
#   lista_temperatura.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Temperatura}).astype(float).corr()['x'].loc['y'])
#   lista_wilgotnosc.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Wilgotnosc}).astype(float).corr()['x'].loc['y'])

# lista_temperatura.sort()
# lista_wilgotnosc.sort()

# fig = make_subplots(rows=1, cols=2, shared_xaxes=False,subplot_titles=("Temperatura","Wilgotność"))    

# fig_temperatura = px.scatter(y=lista_temperatura)
# fig_wilgotnosc = px.scatter(y=lista_wilgotnosc)

# fig.add_trace(fig_temperatura['data'][0], row=1, col=1)
# fig.add_trace(fig_wilgotnosc['data'][0], row=1, col=2)

# fig.update_layout( xaxis_title="Index Stanu",yaxis_title="Wynik korelacji", bargroupgap=0.1)

# fig.show()


In [0]:
lista_temperatura=[]
lista_wilgotnosc=[]
for i in range(len(USAdata)):
  t=USAdata.loc[i]
  lista_temperatura.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Temperatura[0:6]}).astype(float).corr()['x'].loc['y'])
  lista_wilgotnosc.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Wilgotnosc[0:6]}).astype(float).corr()['x'].loc['y'])

lista_temperatura.sort()
lista_wilgotnosc.sort()

fig = make_subplots(rows=1, cols=2, shared_xaxes=False,subplot_titles=("Temperatura","Wilgotność"))    

fig_temperatura = px.histogram(x=lista_temperatura,nbins=30)
fig_wilgotnosc = px.histogram(x=lista_wilgotnosc,nbins=30)

fig.add_trace(fig_temperatura['data'][0], row=1, col=1)
fig.add_trace(fig_wilgotnosc['data'][0], row=1, col=2)

fig.update_layout( yaxis_title="Ilość korelacji", bargroupgap=0.1)

fig.show()

In [0]:

fig = px.scatter_3d(
    x=USAcumdata['Temperatura'].tolist(), 
    y=USAcumdata['Wilgotnosc'].tolist(),
    z=USAcumdata['Przyrosty'].tolist(),
    labels={'x':'Temperatura', 'y':'Wilgotnosc','z':'Przyrosty'}
    )
fig.show()